In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from card_detector import CardDetector, fit
from dataloader_utils import MTGCardsDataset, get_transform_pipe
from config import *

In [ ]:
anchor_boxes = torch.Tensor([[198.27963804, 206.74086672],
       [129.59395666, 161.90171490],
       [161.65437828, 232.34624509]
]) # Anchor boxes acquired from k-means clustering of the dataset

model = CardDetector(img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
                      anchor_boxes=anchor_boxes,
                        num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
                          loss_fn=nn.L1Loss(reduction="sum")
                          )

dummy_image = torch.randn(1, 3, CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]) # create a dummy image to pass to the model and get feature map size
feature_map_shape = model.feature_extractor(dummy_image).shape
feature_map_dims = (feature_map_shape[-2], feature_map_shape[-1])


transform_pipe = get_transform_pipe(img_w=CONFIG["dataset"]["img_w"], img_h=CONFIG["dataset"]["img_h"])

train_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=1,
  transform=transform_pipe,
  limit=13
)
train_dataloader = DataLoader(train_dataset, batch_size=CONFIG["dataloader"]["batch_size"])

val_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=1,
  transform=transform_pipe,
  limit=13
)
val_dataloader = DataLoader(val_dataset, batch_size=CONFIG["dataloader"]["batch_size"]) 

In [ ]:
opt = torch.optim.Adam(params=model.parameters(), lr=CONFIG["optimizer"]["lr"])
fit(model=model, num_epochs=30, optimizer=opt, train_dataloader= train_dataloader, val_dataloader= val_dataloader, device= ("cuda" if torch.cuda.is_available() else "cpu"))

In [ ]:
import torchvision.transforms.functional as fn
from torchvision.io import read_image, ImageReadMode
from torchvision.utils import draw_bounding_boxes


test_image = read_image("data/images_2/1_2.png", mode=ImageReadMode.RGB)
test_transform_pipe = get_transform_pipe(640, 640)
model_input = test_transform_pipe(test_image)

model = model.to("cpu")
pred_box = model.predict(model_input)#, ground_truth=feature_label.unsqueeze(0))
pred_box = pred_box.squeeze(0)
print(pred_box)

#test_image = fn.convert_image_dtype(image=test_image, dtype=torch.uint8)
bbox_img_tensor = draw_bounding_boxes(image=test_image, boxes=pred_box, width=3, colors=(255,255,0))

fn.to_pil_image(bbox_img_tensor)